In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
import tiktoken

from openai.embeddings_utils import get_embedding

In [4]:
datafile_path = "./data/result.json"


In [5]:
df = pd.read_json(datafile_path)


# df = pd.read_csv(datafile_path)
# df["embedding"] = df.embedding.apply(eval).apply(np.array)

In [6]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [7]:
messages_df_from_records = pd.DataFrame.from_records(df.messages.values)

In [8]:
text_entities_df_from_records = pd.DataFrame.from_records(messages_df_from_records.text_entities.values)

In [9]:
def text_entities_get_keys(text_entities: list) -> list:
    keys = set()
    values = []
    for dic in text_entities:
         for key, value in dic.items():
            keys.add(key)
            if key == 'user_id':
                print(text_entities, value)
    return keys, values

total = set()
for val,vale1 in messages_df_from_records.text_entities.apply(text_entities_get_keys).values:
    total = total.union(val)
    if val == 'user_id':
        print(vale1)

[{'type': 'mention_name', 'text': 'Mayank', 'user_id': 565286338}, {'type': 'plain', 'text': ' thats why I was saying the junior programmers are in trouble'}] 565286338
[{'type': 'mention_name', 'text': 'Mayank AIAP', 'user_id': 565286338}] 565286338
[{'type': 'mention_name', 'text': 'Mayank AIAP', 'user_id': 565286338}, {'type': 'plain', 'text': ' : \nso this color would work, but not without the unsqueze:\nimg_pil = '}, {'type': 'link', 'text': 'Image.open'}, {'type': 'plain', 'text': '(imagefp)\nimg_tensor = pil_to_tensor(img_pil)\nprint(img_tensor.shape)\n\nedge_filter = torch.tensor(\n    [[[[ 0,0,0],\n       [ 0,1,0 ],\n       [ 0,0,0]]]]).float()\n\nimg_tensor_ = F.conv2d(img_tensor.unsqueeze(1), edge_filter)\nprint(img_tensor_.shape)\n\nimg_tensor_.clamp_min_(0).clamp_max_(1)\nprint(img_tensor_.shape)\n\nimg = img_tensor_[:,0,:,:]\ntensor_to_pil(img)'}] 565286338
[{'type': 'mention_name', 'text': 'Shu Ying', 'user_id': 5362962200}] 5362962200
[{'type': 'mention_name', 'text': '

In [10]:
def text_entities_get_type(text_entities: list) -> list:
    types = set()
    val = ''
    for dic in text_entities:
        for key, value in dic.items():
            if key == 'type':
                val = value
    if val == 'bot_command':
        print(text_entities)
    return val

total_types = set()
for val in messages_df_from_records.text_entities.apply(text_entities_get_type).values:
    total_types = total_types | set([val])


[{'type': 'bot_command', 'text': '/start_roll_call'}]
[{'type': 'bot_command', 'text': '/in'}]
[{'type': 'bot_command', 'text': '/in@WhosInBot'}]


In [11]:
def parse_text_entities(text_entities: list) -> str:
    text = ''
    for dic in text_entities:
        if dic.get('type') == 'plain':
            text += dic.get('text')
    return text

text_df = messages_df_from_records.text_entities.apply(parse_text_entities)


In [12]:
text_df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 350 entries, 0 to 349
Series name: text_entities
Non-Null Count  Dtype 
--------------  ----- 
350 non-null    object
dtypes: object(1)
memory usage: 2.9+ KB


In [13]:
text_df.sample(10)

34                                          class? haha.
51     Yup, I started that Discord server. You can us...
232    Just found this for gradient ascent:\n\n(if yo...
220                                          I signed up
267               dat one is not the haunted one right 😂
200                                                     
24                                                     .
254                                           NEW LINK: 
156                                                     
246                                          much easier
Name: text_entities, dtype: object

In [14]:
text_df = text_df[text_df != '']

In [15]:
text_df = pd.DataFrame(text_df)

In [16]:
text_df

,text_entities
5,hello i am marvin ng
7,Your username ah.. Likely scam leh. 😂
8,Hahaha
10,"Hihi, where did u guys went"
11,Arts canteen .. the one we went yesterday
...,...
340,In case you haven't had enough of transformers...
345,WEF summit talks. Multi day event one
346,"where u sia, i tot u came back before us"
347,Toilet


In [17]:
# load & inspect dataset


text_df["text_entities"] = (
    text_df.text_entities.str.strip() 
)
text_df.sample(10)

,text_entities
129,FYI someone from my Discord server did a ChatG...
217,i am game for this
280,Video call link:
47,
254,NEW LINK:
313,meant to save it to my saved messages
97,"hi guys, cause i had some weird issue yesterda..."
51,"Yup, I started that Discord server. You can us..."
52,Should there be individual channels for batche...
96,Suggested resource creator for MLOps (suggeste...


In [18]:

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
text_df["n_tokens"] = text_df.text_entities.apply(lambda x: len(encoding.encode(x)))
text_df = text_df[text_df.n_tokens <= max_tokens]
len(text_df)

278

In [23]:
text_df = text_df.loc[text_df['text_entities'] != ""]

In [24]:
# text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 275 entries, 5 to 348
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text_entities  275 non-null    object
 1   n_tokens       275 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


In [25]:
# This may take a few minutes
text_df["embedding"] = text_df.text_entities.apply(lambda x: get_embedding(x, engine=embedding_model))
text_df.to_csv("data/telegram_v1_with_embeddings.csv")

C:\Users\y3kma\AppData\Local\Temp\ipykernel_21076\4249027083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df["embedding"] = text_df.text_entities.apply(lambda x: get_embedding(x, engine=embedding_model))


In [26]:
text_df

,text_entities,n_tokens,embedding
5,hello i am marvin ng,6,"[-0.02315955050289631, -0.0253192987293005, -0..."
7,Your username ah.. Likely scam leh. 😂,11,"[0.006359778344631195, -0.01984380930662155, 0..."
8,Hahaha,2,"[-0.01198045164346695, -0.021351421251893044, ..."
10,"Hihi, where did u guys went",8,"[-0.005862023681402206, -0.0022562702652066946..."
11,Arts canteen .. the one we went yesterday,11,"[-0.005921371281147003, 0.001968239899724722, ..."
...,...,...,...
340,In case you haven't had enough of transformers...,14,"[-0.005566476844251156, -0.031638238579034805,..."
345,WEF summit talks. Multi day event one,9,"[-0.022343982011079788, -0.014287988655269146,..."
346,"where u sia, i tot u came back before us",11,"[0.020630987361073494, -0.020216763019561768, ..."
347,Toilet,2,"[0.0015059447614476085, -0.0001866410311777144..."
